In [30]:
# import the relevant libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [31]:
data = pd.read_csv('cleaned_car_data.csv')
data.head()

,name,make,year,age,mileage,transmission,colour,price
0,2000 BUICK CENTURY CUSTOM Sedan 4 Door,BUICK,2000,21,1.0,AUTOMATIC,CHAMPAGNE,357
1,2004 HONDA CIVIC LX,HONDA,2004,17,134095.0,AUTOMATIC,GRAY,850
2,1993 FORD MUSTANG LX,FORD,1993,28,99086.0,AUTOMATIC,WHITE,975
3,1998 HONDA CR-V LX,HONDA,1998,23,194018.0,AUTOMATIC,BLUE,925
4,1999 CHEVROLET TAHOE K1500 Wagon 4 Door,CHEVROLET,1999,22,264054.0,AUTOMATIC,GRAY,750


In [32]:
#drop the name column because it is irrelevant in our model building
data = data.drop('name', axis=1)

In [33]:
data.head()

,make,year,age,mileage,transmission,colour,price
0,BUICK,2000,21,1.0,AUTOMATIC,CHAMPAGNE,357
1,HONDA,2004,17,134095.0,AUTOMATIC,GRAY,850
2,FORD,1993,28,99086.0,AUTOMATIC,WHITE,975
3,HONDA,1998,23,194018.0,AUTOMATIC,BLUE,925
4,CHEVROLET,1999,22,264054.0,AUTOMATIC,GRAY,750


In [34]:
# get dummny data
data = pd.get_dummies(data)
data

,year,age,mileage,price,make_ACURA,make_AUDI,make_BMW,make_BUICK,make_CADILLAC,make_CHEVROLET,...,colour_MAROON,colour_ORANGE,colour_PURPLE,colour_RED,colour_SILVER,colour_TAN,colour_TEAL,colour_TWO TONE,colour_WHITE,colour_YELLOW
0,2000,21,1.0,357,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2004,17,134095.0,850,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1993,28,99086.0,975,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1998,23,194018.0,925,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1999,22,264054.0,750,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,2006,15,182744.0,450,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
905,2006,15,239054.0,950,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
906,1997,24,220043.0,800,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
907,2002,19,0.0,1000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
